In [ ]:
# packages = ('pytorch_lightning timm==0.6.12 ipywidgets==7.7.1 opencv-python zstandard awscli ' 
#                     +  'transformers librosa torchlibrosa torchaudio torchvision ' 
#                     + 'lion-pytorch segmentation-models-pytorch==0.3.2 ' 
#                     'fcwt') # fsspec[s3] albumentations==1.3.0 
# !pip install -q -U $packages

In [ ]:
# FAST = False
SAMPLE = 4

In [ ]:
# DATA_PATH = '/data/'
DATASET = 'walkdata4'
TAG = 'tlvmc-parkinsons-freezing-gait-prediction'
INPUT_PATH = '/kaggle/input/' + TAG
DATA_PATH = '/tmp/'
CODE_PATH = '/kaggle/input/' + DATASET
PREFIX = 'walk/'
OFFLINE = True

In [ ]:
!ls $CODE_PATH

In [ ]:
WHEEL_PATH = CODE_PATH + '/whl'
import os
for p in os.listdir(WHEEL_PATH):
    if 'segment' in p: continue; 
    !pip install {WHEEL_PATH}/{p}

In [ ]:
import sys
sys.path.append(CODE_PATH + '/smp')
# import segmentation_models_pytorch as smp
# smp.Unet(encoder_weights = None);

In [ ]:
import zipfile
import boto3
s3 = boto3.client('s3')

import os
import io
from joblib import Parallel, delayed

import json
import pickle
import zstandard as zstd
zd = zstd.ZstdDecompressor()
zc = zstd.ZstdCompressor()

import math
import random
import datetime

In [ ]:
# walk and list all files
objs = []
for root, path, files in os.walk(INPUT_PATH):
    if 'train' in root: continue
    objs.extend([os.path.join(root, f).replace(INPUT_PATH + '/', '') for f in files])
    print(root, len(files))
len(objs)

In [ ]:
def getFile(file, save = True):
#     s3 = boto3.client('s3')
#     zd = zstd.ZstdDecompressor()
#     file_path = DATA_PATH + 'data/' + file + '.zstd'
#     if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
#         if save: os.makedirs(os.path.dirname(file_path), exist_ok=True)
#         fin = s3.get_object(Bucket = DATA_BUCKET, Key = PREFIX + 'data/' + file + '.zstd')['Body']#.read()        
#         if save:
#             with open(file_path, 'wb') as f: f.write(fin.read())
    file_path = os.path.join(INPUT_PATH, file)# + '.zstd'
    out = io.BytesIO()
    if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
        with open(file_path, 'rb') as fin:
#             zd.copy_stream(fin, out)
#     else:        
            out.write(fin.read())
    out.seek(0)
    # print(out.getbuffer().nbytes)
    return out

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7, 4)

from IPython.display import display
np.random.seed(datetime.datetime.now().microsecond)

In [ ]:
files = sorted(objs)#o)[0] for o in objs])
display(files[:10])

events = pd.read_csv(getFile('events.csv'))
subjects = pd.read_csv(getFile('subjects.csv'))
tasks = pd.read_csv(getFile('tasks.csv'))
daily_metadata = pd.read_csv(getFile('daily_metadata.csv'))
defog_metadata = pd.read_csv(getFile('defog_metadata.csv'))
tdcsfog_metadata = pd.read_csv(getFile('tdcsfog_metadata.csv'))
sample = pd.read_csv(getFile('sample_submission.csv'), index_col = 0, 
                    ).astype(np.float32)

In [ ]:
def prep_metadata(defog_metadata, tdcsfog_metadata, daily_metadata, subjects, full = True,
                    expanded = True):
    m1 = defog_metadata.copy()
    m1.insert(3, 'Test', 0)
    m1 = m1.merge(subjects, on = ['Subject', 'Visit', ], how = 'inner')
    assert len(m1) == len(defog_metadata)

    m2 = tdcsfog_metadata.copy()
    m2 = m2.merge(subjects.drop(columns = 'Visit'), on = ['Subject', ], how = 'inner')
    assert len(m2) == len(tdcsfog_metadata)

    m3 = daily_metadata.copy()
    m3 = m3.merge(subjects, on = ['Subject', 'Visit' ], how = 'inner')
    m3.insert(3, 'Test', 0)
    m3.insert(4, 'Medication', (defog_metadata.Medication == 'on').mean())
    m3.drop(columns = [c for c in m3.columns if 'recording' in c], inplace = True)
    assert len(m3) == len(daily_metadata)

    metadata = pd.concat([m1, m2, #m3
                            ], axis = 0)
    metadata.Medication = 1 * (metadata.Medication == 'on')
    metadata.Sex = 1 * (metadata.Sex == 'M')
    
    if expanded:
        metadata['num_tests'] = metadata.groupby('Subject').transform(lambda x: x.nunique()).Test
        metadata['max_visit'] = metadata.groupby('Subject').transform(lambda x: x.max()).Visit
        metadata['visit_medications'] = metadata.groupby(['Subject', 'Visit']).transform('nunique').Medication 
        metadata['UPDRS_On_vs_Off'] = metadata.UPDRSIII_On - metadata.UPDRSIII_Off
        # add 4 columns to dmetadata
    

    if full:
        # null fix
        metadata['Uon_null'] = 1 * (metadata.UPDRSIII_On.isnull())
        metadata['Uoff_null'] = 1 * (metadata.UPDRSIII_Off.isnull())
        metadata.UPDRSIII_On = metadata.UPDRSIII_On.fillna(metadata.UPDRSIII_On.mean())
        metadata.UPDRSIII_Off = metadata.UPDRSIII_Off.fillna(metadata.UPDRSIII_Off.mean())
        if expanded:
            metadata.UPDRS_On_vs_Off = metadata.UPDRS_On_vs_Off.fillna(metadata.UPDRS_On_vs_Off.mean())   

    metadata.set_index('Id', inplace = True)
    
    if full:
        metadata['Test_Nonzero'] = 1. * (metadata.Test > 0)
        # for i in range(1):
        #     metadata['Test{}'.format(i)] = metadata.Test == i
        metadata.iloc[:, 1:] = metadata.iloc[:, 1:].astype(np.float32)
        metadata.iloc[:, 1:] = (metadata.iloc[:, 1:] - metadata.iloc[:, 1:].mean(0)) / metadata.iloc[:, 1:].std(0)  
        metadata.iloc[:, 1:] = metadata.iloc[:, 1:].clip(-3, 3)

    msubject = metadata.Subject
    metadata.drop(columns = 'Subject', inplace = True)
    if full:
        metadata = metadata.astype(np.float32)
    
    m3 = m3.set_index('Id')
    assert m3.shape[1] <= metadata.shape[1]; i = 0
    while m3.shape[1] < metadata.shape[1]:
        m3.insert(m3.shape[1], 'dummy_{}'.format(i), 0); i += 1
    m3.iloc[:, -1] = metadata.iloc[:, -1].min() # yes, hack, for default_metadata in dataset.py 

    return metadata, msubject, m3

In [ ]:
fog_files = sorted([f for f in files if 'test/' in f and 'fog' in f])
# if FAST and len(fog_files) == 2:
#     fog_files = [f for f in fog_files if 'tdcs' in f]
print(len(fog_files))

unlabeled_files = [f for f in files if 'unlabeled/' in f]# and not any([z in f for z in common_files])]
print(len(unlabeled_files))

In [ ]:
# load and process data
def load(f):
    return np.load(DATA_PATH + 'cache/' + f + '.npy')

def process(f):
    # if exists, return stats
    cache_file = DATA_PATH + 'cache/' + f + '.npy'
    if os.path.exists(cache_file) and os.path.getsize(cache_file) > 0: 
        return load(f).shape, os.path.getsize(cache_file)
    
    # if not, load array,
    df = pd.read_csv(getFile(f, ), )
#     print(df)
    assert (df.index == df.Time).all()
    # verify
    assert (df.Time == df.index).all()
    assert all(df.columns == ['Time',
                           'AccV', 'AccML', 'AccAP',
                           'StartHesitation', 'Turn', 'Walking',
                            'Valid', 'Task'][:len(df.columns)])
    if 'train/' not in f: assert len(df.columns) in [4]
    
    v = np.zeros((len(df), 12), dtype = np.float32)
    v[:, 6:8] = 1
    v[:, :df.shape[1] - 1] = df.iloc[:, 1:]

#     fid = f.split('/')[-1].split('.')[0]
#     mult = 100 if 'tdcs' not in f else 128
#     for e in events[events.Id == fid].itertuples():
#         v[int(round(e.Init * mult)): int(round(e.Completion * mult)), 8] = 1
#         v[int(round(e.Init * mult)): int(round(e.Completion * mult)), 9] = e.Kinetic
#         v[int(round(e.Init * mult)): int(round(e.Completion * mult)), 10] = 1 - e.Kinetic
#     for e in tasks[tasks.Id == fid].itertuples():
#         v[int(round(e.Begin * mult)): int(round(e.End * mult)), 11] = task_dict[e.Task]

    # store as compresssed;
    assert v.dtype == np.float32
    # zc = zstd.ZstdCompressor()
    # compr = zc.compress(pickle.dumps(v))
    os.makedirs(os.path.dirname(cache_file), exist_ok = True)
    np.save(cache_file, v)
    # with open(cache_file, 'wb') as f:
    #     f.write(compr)

    return v.shape, os.path.getsize(cache_file)#len(compr)


In [ ]:
# fog_files = sorted(fog_files)[-1:]
# fog_files # ***

In [ ]:

# process all files
r = Parallel(os.cpu_count())(delayed(process)(f) for f in fog_files[:])

# display counts;
fogcount = dict(zip(fog_files, [e[0][0] for e in r]))
[sum([v for k, v in fogcount.items() if s in k]) / 1e6
        for s in ['/defog/', '/tdcsfog/', ]]

In [ ]:
!du -sh {DATA_PATH}cache/test*

In [ ]:
%run -i {CODE_PATH}/dataset.py

In [ ]:
%run -i {CODE_PATH}/model.py

In [ ]:
def getData(params, m, lcount, metadata):
    test_data = WalkDataset(
        {k: v for k, v in lcount.items()},
                      metadata, load, None, -1,
                      test = True,
                      **getParams(WalkDataset, params))
    test_loader = DataLoader(test_data, batch_size = 32,
                              num_workers = os.cpu_count())
    print(len(test_data), len(test_loader))
    return test_data, test_loader

In [ ]:
def getModel(params, m):
    model = WalkNetwork(params, **getParams(WalkNetwork, params)).to(device)
    model.load_state_dict(
        pickle.load(open(CODE_PATH + '/models/' + m + '.pt', 'rb')), 
    )
    model.to(device);
    model.eval();
    return model

In [ ]:
def infer(model, test_loader):
    yps, ss, fs, idxs = [], [], [], []
    for batch in test_loader:
        x, y, s, frac, m_, f, i, flen = batch
        with torch.no_grad():
            yp = model(*[e.to(device)
                                     for e in [x, m_, frac, flen]], 
                                 )[0]
        yps.append(yp[..., :3].cpu())
        ss.append(s.cpu())
        fs.append(f)
        idxs.append(i.cpu())
#         print(yp.shape)
    
    # process
#     n_models = yps[0].shape[-2]
#     print(n_models)
    
    fs = np.concatenate(fs)
    idxs = torch.cat(idxs)
    slen = yps[0].shape[1]
    s = torch.cat(ss, 0).reshape(-1, slen, 1).cpu().numpy()
    yps = torch.cat(yps, 0)
    yps = yps.reshape(-1, slen, 3).cpu().numpy()        
    print(yps.shape, s.shape, len(fs), len(idxs))
    assert all([len(e) == len(yps) for e in [fs, idxs, s, yps]])

    pred_dict, ct_dict, target_dict = {}, {}, {}
    for f in list(set(fs)):
        l = idxs[fs == f].max() + slen // SAMPLE
        pred_dict[f] = np.zeros((l, 3), dtype = np.float32)
        ct_dict[f] = np.zeros((l, 3), dtype = np.float32)

    for i in range(len(yps)):
        pred_dict[fs[i]][idxs[i]:idxs[i] + slen] += yps[i] * s[i]
        ct_dict[fs[i]][idxs[i]:idxs[i] + slen] += s[i]

    # compile and nrm
    spred_dict = {}; final_yps = []
    for k in ct_dict:
        print(ct_dict[k].shape)
        f = ct_dict[k].sum(1) > 0
        spred_dict[k] = pred_dict[k][f] / (ct_dict[k][f] + 1e-5)
        final_yps.append(spred_dict[k])

    final_yps = np.concatenate(final_yps)
    avg = final_yps.mean(0)
    print(avg)
    
    spred_dict = {k: v/avg for k, v in spred_dict.items()}
    return spred_dict
    

In [ ]:
def inferParallel(model, test_loader):
    yps, ss, fs, idxs = [], [], [], []
    for batch in test_loader:
        x, y, s, frac, m_, f, i, flen = batch
        with torch.no_grad():
            yp = model(*[e.to(device)
                                     for e in [x, m_, frac, flen]], 
                                 )#[0]
#         SAMPLE = 10
        yps.append(yp[:, ::SAMPLE, :, :3].cpu())
        ss.append(s[:, ::SAMPLE].cpu())
        fs.append(f)
        idxs.append((i / SAMPLE).long().cpu())
#         print(yp.shape)
    
    # process
    n_models = yps[0].shape[-2]
    print(n_models)
    
    fs = np.concatenate(fs)
    idxs = torch.cat(idxs)
    slen = yps[0].shape[1]
    s = torch.cat(ss, 0).reshape(-1, slen, 1, 1).cpu().numpy()
    yp = torch.cat(yps, 0); del yps
    yps = yp.reshape(-1, slen, n_models, 3).cpu().numpy(); del yp        
    print(yps.shape, s.shape, len(fs), len(idxs))
    assert all([len(e) == len(yps) for e in [fs, idxs, s, yps]])

    pred_dict, ct_dict, target_dict = {}, {}, {}
    for f in list(set(fs)):
        l = idxs[fs == f].max() + slen
        pred_dict[f] = np.zeros((l, n_models, 3), dtype = np.float32)
        ct_dict[f] = np.zeros((l, n_models, 3), dtype = np.float32)

    for i in range(len(yps)):
        pred_dict[fs[i]][idxs[i]:idxs[i] + slen] += yps[i] * s[i]
        ct_dict[fs[i]][idxs[i]:idxs[i] + slen] += s[i]

    # compile and nrm
    spred_dict = {}; final_yps = []
    for k in ct_dict:
        print(ct_dict[k].shape)
        f = ct_dict[k].sum(2).sum(1) > 0
        spred_dict[k] = pred_dict[k][f] / (ct_dict[k][f] + 1e-5)
        final_yps.append(spred_dict[k])

    final_yps = np.concatenate(final_yps) # B, M, C
    avg = final_yps.mean(0)  # M, C
    print(avg)
#     avg = avg / avg.mean(0, keepdims = True) # each class should sum to 1;
#     print(avg)
#     print((k/avg).sum(0))
    
    spred_dict = {k: (v/avg).mean(-2) for k, v in spred_dict.items()}
    return spred_dict
    

In [ ]:
# ms = [f.split('.')[0]#, json.load(open(CODE_PATH + '/params/' + f)))
#               for f in os.listdir(CODE_PATH + '/params')]
# print(len(ms))

In [ ]:
ms = [(f.split('.')[0], json.load(open(CODE_PATH + '/params/' + f)))
              for f in os.listdir(CODE_PATH + '/params')]
print(len(ms))

In [ ]:
select_ms = '0c1908, 38f97e, 6724d4, b904fe, 4e7844, b792a2, d64c60, d85bf1, 798bc8, c5be66, cd590d, d7b7b6, 2233bd, 2c1d4a, b42aae, ef4805, f41ae6, 18da64, 37898f, 9805ce, f65f09, 53b767, 7033ef, 90be5b, fdbb0a, 36b30a, 94cae9, cbf5ad, fdd535, 25f680, b8cf10, c8d569, faed9c, 91cee9, a33a0c, a4745d, a64923, c89baf, 1ac968, 22693b, e1d1a2, fc8f37'
select_ms = select_ms.split(', ')

In [ ]:
from collections import defaultdict
mgroups = defaultdict(list)
mpoints = defaultdict(int)
for m, p in ms:
    if m not in select_ms: continue
    # if p.get('xformer_init_1') == 0.1: continue;
    # if p.get('xformer_layers', 2) > 3: continue;
    # if p['frac_pwr_mult'] != f: continue
    # if 'seg' in p: continue# or 'melspec' in p: continue
    # if p['batch_size'] >= 20: continue;
    r_ = {k: v for k, v in p.items() 
              if k not in ['fold', 'seed' ]#'seed', 'n_folds' ]
         }
    mgroups[json.dumps(r_)].append(m)
    mpoints[json.dumps(r_)] += 1 - select_ms.index(m)/len(select_ms)
len(mgroups)

In [ ]:
plt.hist(mpoints.values(), bins =50);

In [ ]:
sum(mpoints.values())

In [ ]:
class ParallelModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)
    
    def forward(self, x, m, frac, flen):
        return torch.stack([model(x, m, frac, flen)[0]
                            for model in self.models], -2)


In [ ]:
# numpy add two arrays, expanding the smaller one on axis 0
def eadd(a, b):
    if isinstance(a, int): return b.copy();
    if a.shape[0] < b.shape[0]:
        a = np.pad(a, ((0, b.shape[0] - a.shape[0]), (0, 0)), mode = 'constant')
    elif a.shape[0] > b.shape[0]:
        b = np.pad(b, ((0, a.shape[0] - b.shape[0]), (0, 0)), mode = 'constant')
    return a + b

In [ ]:
import time

In [ ]:
all_pred_dict = {}; ct = 0
# for m in ms:#_, 
for _, ms in mgroups.items():
    models = []; m = ms[0];# print(json.loads(_))
    params = json.load(open(CODE_PATH + '/params/' + m + '.json', 'r'))
#     if params['frac_pwr_mult'] not in [2.36, 2.04, 1.81, 1.91, 1.88]: continue
    for m in ms:
        models.append(getModel(params, m ))    
    model = ParallelModel(models).to(device).eval()
    metadata, msubject, md = prep_metadata(defog_metadata, tdcsfog_metadata, daily_metadata, subjects,
                                      expanded = params['expanded'])
#     model = getModel(params, m)
    test_data, test_loader = getData(params, m, fogcount, metadata)
    start = time.time()
    spred_dict = inferParallel(model, test_loader)
#     spred_dict = infer(model, test_loader)
    print(time.time() - start)
    
    for k, v in spred_dict.items():
        all_pred_dict[k] = eadd(all_pred_dict.get(k, 0), 
                                v * mpoints[_] ** 0.8)
    del test_data, test_loader, spred_dict
        
    ct += 1;
    print()
    del model#, models


In [ ]:
preds = []
for k, v in all_pred_dict.items():
    true_len = len(load(k))
    print(k, len(v), true_len)
    v = cv2.resize(v, None, fx = 1, fy = SAMPLE)
    v = v[:true_len]
    v = np.pad(v, ((0, true_len - len(v) ), (0, 0)), mode = 'edge')
    
    df = pd.DataFrame(v / ct, columns = sample.columns, 
                index = ['{}_{}'.format(k.split('/')[-1].split('.')[0], i)
                             for i in range(len(v))],
                     dtype = np.float32)
    df.index.name = 'Id'
    preds.append(df)
preds = pd.concat(preds)#.reset_index()
print(len(preds))
preds.tail()

In [ ]:
if len(fog_files) == 2:
    preds = preds[:-1]#.head(200)

In [ ]:
if len(preds) != len(sample): 
    preds = pd.concat((preds.reindex(
        sorted(list(set(sample.index) & set(preds.index))) ), 
                    sample.reindex(
        sorted(list(set(sample.index) - set(preds.index))) ) ))
len(preds)

In [ ]:
preds.tail(5)

In [ ]:
assert len(preds) == len(sample) and all(preds.columns == sample.columns)
assert sorted(preds.index.tolist()) == sorted(sample.index.tolist())

In [ ]:
preds.to_csv('submission.csv')#, index = False)

In [ ]:
preds2 = pd.read_csv('submission.csv')
sample2 = pd.read_csv(getFile('sample_submission.csv'))
assert len(preds2) == len(sample2) and all(preds2.columns == sample2.columns)
assert sorted(preds2.index.tolist()) == sorted(sample2.index.tolist())

In [ ]:
for k in fog_files[:2]:
    plt.figure()
    plt.plot(all_pred_dict[k] / ct)

In [ ]:
np.concatenate(list(all_pred_dict.values())).mean(0)